In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!apt-get install openjdk-8-jre
!apt-get install scala
!pip install py4j
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jre is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
scala is already the newest version (2.11.12-4~18.04).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
!rm -r *.tgz *.zip sample_data
!ls


rm: cannot remove '*.zip': No such file or directory
rm: cannot remove 'sample_data': No such file or directory
drive  spark-3.1.2-bin-hadoop3.2


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/CS5344/GP')


In [ ]:
# output operating time
!pip install ipython-autotime
%load_ext autotime

time: 157 µs (started: 2022-03-20 11:32:12 +00:00)


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import math
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel, HashingTF, IDF, IDFModel

# spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder.appName("TestSetup").getOrCreate()
sc = spark.sparkContext

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yishun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/yishun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yishun/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/yishun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
env: bash: No such file or directory


RuntimeError: Java gateway process exited before sending its port number

## Input 2 labeled datasets and combine them together
### Output: 
- label_data (dataframe, column=['id', 'label', 'text'])

In [ ]:
# input the first labeled dataset
origin_Fake = pd.read_csv('/Users/yishun/Desktop/BAModule/CS5344/group project/Fake.csv')
origin_True = pd.read_csv('/Users/yishun/Desktop/BAModule/CS5344/group project/True.csv')
# print(origin_Fake.head())
# print(origin_True.head())

# keep the text features and generate the labels
origin_Fake['text'] = origin_Fake.apply(lambda row: row['title']+" "+row['text'], axis=1)
origin_True['text'] = origin_Fake.apply(lambda row: row[0]+row[1], axis=1)
origin_Fake['label'] = 0 # Fake
origin_True['label'] = 1 # True
label_data1 = pd.DataFrame(columns=['text', 'label'])
label_data1 = label_data1.append(origin_Fake[['text','label']])
label_data1 = label_data1.append(origin_True[['text','label']])

# remove null data
label_data1 = label_data1.dropna()
# print(label_data1.head())
print("# of records of the first text datasets:", len(label_data1))

# of records of the first text datasets: 44898
time: 2.32 s (started: 2022-03-20 11:32:19 +00:00)


In [ ]:
# input the second labeled dataset
origin_train = pd.read_csv('/Users/yishun/Desktop/BAModule/CS5344/group project/train.csv')
# origin_test = pd.read_csv('/content/drive/My Drive/CS5344/GP/test.csv') # No label
# origin_train = origin_train.append(origin_test)
# print(origin_train)

# keep the text features and remove null data
origin_train = origin_train[['title','text','class']]
origin_train = origin_train.dropna()
origin_train['text'] = origin_train.apply(lambda row: row['title']+" "+row['text'], axis=1)
origin_train = origin_train.replace({'Fake':0, 'Real':1})
origin_train = origin_train.rename(columns={'class':'label'})
label_data2 = origin_train[['text','label']]
# print(label_data2.head())
print("# of records of the second text datasets:", len(label_data2))

# of records of the second text datasets: 40000
time: 2.16 s (started: 2022-03-20 11:32:21 +00:00)


In [ ]:
# combine the labeled datasets together
label_data = label_data1.append(label_data2)
label_data['id'] = list(range(len(label_data))) 
label_data = label_data[['id','text','label']]
print(label_data.head())
print("# of records of the labeled text dataset:", len(label_data))

   id                                               text label
0   0   Donald Trump Sends Out Embarrassing New Year’...     0
1   1   Drunk Bragging Trump Staffer Started Russian ...     0
2   2   Sheriff David Clarke Becomes An Internet Joke...     0
3   3   Trump Is So Obsessed He Even Has Obama’s Name...     0
4   4   Pope Francis Just Called Out Donald Trump Dur...     0
# of records of the labeled text dataset: 84898
time: 47.3 ms (started: 2022-03-20 11:32:23 +00:00)


## Input the unlabeled dataset
### Output: 
- main_data (dataframe, column=['id', 'text'])

In [ ]:
# input the unlabeled dataset
origin_main = pd.read_csv('/Users/yishun/Desktop/BAModule/CS5344/group project/fake_news_dataset.csv')
# print(origin_main.head())

# exclude other language
origin_main = origin_main[origin_main['language']=='english'] 


# keep the text features and remove null data
origin_main = origin_main[['uuid','title','text']]
origin_main = origin_main.rename(columns={'uuid':'id'})
origin_main = origin_main.dropna()
origin_main['text'] = origin_main.apply(lambda row: row['title']+" "+row['text'], axis=1)
main_data = origin_main[['id','text']]
# print(main_data['id'].unique()) # the id is unique
print(main_data.head)
print("# of records of the main text dataset:", len(main_data))

<bound method NDFrame.head of                                              id  \
0      6a175f46bcd24d39b3e962ad0f29936721db70db   
1      2bdc29d12605ef9cf3f09f9875040a7113be5d5b   
2      c70e149fdd53de5e61c29281100b9de0ed268bc3   
3      7cf7c15731ac2a116dd7f629bd57ea468ed70284   
4      0206b54719c7e241ffe0ad4315b808290dbe6c0f   
...                                         ...   
12908  613e1a6e130b5a3f5df62c8fb0b73667742a43db   
12909  dedc36a34e5cb1062bf4627d314227f60cd9a708   
12910  cd8bb1ae426287f3a63c2979b3b5dfb0277b10e2   
12911  213eb9eeb5479ad2588b54b24acd53bc8ead8e8c   
12912  795f05dce10c27ff4e7b3f39ddf4e75075f5421c   

                                                    text  
0      Muslims BUSTED: They Stole Millions In Gov’t B...  
1      Re: Why Did Attorney General Loretta Lynch Ple...  
2      BREAKING: Weiner Cooperating With FBI On Hilla...  
3      PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...  
4      FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...  
...

## Data Preprocessing
### Steps:
- convert to lowercase
- remove numbers
- remove puctuations
- remove stopwords
- lemmatization

### Output: 
- label_data_df (spark dataframe, column=['id', 'label', 'token'])
- main_data_df (spark dataframe, column=['id', 'token'])

In [ ]:
# load the data into rdd format
label_data_rdd = sc.parallelize(label_data.values.tolist())
main_data_rdd = sc.parallelize(main_data.values.tolist())

# transformed to lowercase
label_data_lower = label_data_rdd.map(lambda f: ((f[0], f[2]), str.lower(f[1])))
main_data_lower = main_data_rdd.map(lambda f: (f[0], str.lower(f[1])))
# print(label_data_lower.take(5))

# remove numbers
label_data_no_number = label_data_lower.map(lambda f: (f[0], re.sub(r'\d+', '', f[1])))
main_data_no_number = main_data_lower.map(lambda f: (f[0], re.sub(r'\d+', '', f[1])))
# print(label_data_no_number.take(5))

# remove punctuation and split it into words
label_data_words = label_data_no_number.map(lambda f: (f[0], re.split(r'[^\w]+',f[1])))
label_data_words = label_data_words.map(lambda f: (f[0], [i for i in f[1] if i != '']))
main_data_words = main_data_no_number.map(lambda f: (f[0], re.split(r'[^\w]+',f[1])))
main_data_words = main_data_words.map(lambda f: (f[0], [i for i in f[1] if i != '']))
# print(label_data_words.take(5))


time: 2.15 s (started: 2022-03-20 11:32:24 +00:00)


In [ ]:
# remove stopwords
def remove_stopwords(words, lang='english'):
  from nltk.corpus import stopwords
  lang_stopwords = stopwords.words(lang)
  stopwords_removed = [w for w in words if w not in lang_stopwords]
  return stopwords_removed

label_data_no_stopwords = label_data_words.map(lambda f: (f[0], remove_stopwords(f[1])))
main_data_no_stopwords = main_data_words.map(lambda f: (f[0], remove_stopwords(f[1])))
print(label_data_no_stopwords.take(5))

# lemmatization
# Function to find part of speech tag for a word
def find_pos(word):
    # Part of Speech constants
    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'

    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
    
    # Adjective tags - 'JJ', 'JJR', 'JJS'
    if pos.lower()[0] == 'j':
        return 'a'
    # Adverb tags - 'RB', 'RBR', 'RBS'
    elif pos.lower()[0] == 'r':
        return 'r'
    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
    elif pos.lower()[0] == 'v':
        return 'v'

    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
    else:
        return 'n'


# Function to apply lemmatization to a list of words
def words_lemmatizer(words, encoding="utf8"):
    lemma_words = []
    wl = WordNetLemmatizer()
    for word in words:
        pos = find_pos(word)
        lemma_words.append(wl.lemmatize(word, pos))
    return lemma_words
    

label_data_lemmatization = label_data_no_stopwords.map(lambda f: (f[0][0], f[0][1], words_lemmatizer(f[1])))
main_data_lemmatization = main_data_no_stopwords.map(lambda f: (f[0], words_lemmatizer(f[1])))
# print(label_data_lemmatization.take(5))


[((0, 0), ['donald', 'trump', 'sends', 'embarrassing', 'new', 'year', 'eve', 'message', 'disturbing', 'donald', 'trump', 'wish', 'americans', 'happy', 'new', 'year', 'leave', 'instead', 'give', 'shout', 'enemies', 'haters', 'dishonest', 'fake', 'news', 'media', 'former', 'reality', 'show', 'star', 'one', 'job', 'country', 'rapidly', 'grows', 'stronger', 'smarter', 'want', 'wish', 'friends', 'supporters', 'enemies', 'haters', 'even', 'dishonest', 'fake', 'news', 'media', 'happy', 'healthy', 'new', 'year', 'president', 'angry', 'pants', 'tweeted', 'great', 'year', 'america', 'country', 'rapidly', 'grows', 'stronger', 'smarter', 'want', 'wish', 'friends', 'supporters', 'enemies', 'haters', 'even', 'dishonest', 'fake', 'news', 'media', 'happy', 'healthy', 'new', 'year', 'great', 'year', 'america', 'donald', 'j', 'trump', 'realdonaldtrump', 'december', 'trump', 'tweet', 'went', 'welll', 'expect', 'kind', 'president', 'sends', 'new', 'year', 'greeting', 'like', 'despicable', 'petty', 'infant

In [ ]:
# generate spark df of labeled dataset
label_data_df = spark.createDataFrame(label_data_lemmatization, ['id', 'label', 'token'])
label_data_df.show()

# generate spark df of unlabeled dataset
main_data_df = spark.createDataFrame(main_data_lemmatization, ['id', 'token'])
main_data_df.show()

+---+-----+--------------------+
| id|label|               token|
+---+-----+--------------------+
|  0|    0|[donald, trump, s...|
|  1|    0|[drunk, bragging,...|
|  2|    0|[sheriff, david, ...|
|  3|    0|[trump, obsess, e...|
|  4|    0|[pope, francis, c...|
|  5|    0|[racist, alabama,...|
|  6|    0|[fresh, golf, cou...|
|  7|    0|[trump, say, insa...|
|  8|    0|[former, cia, dir...|
|  9|    0|[watch, brand, ne...|
| 10|    0|[papa, john, foun...|
| 11|    0|[watch, paul, rya...|
| 12|    0|[bad, news, trump...|
| 13|    0|[watch, lindsey, ...|
| 14|    0|[heiress, disney,...|
| 15|    0|[tone, deaf, trum...|
| 16|    0|[internet, brutal...|
| 17|    0|[mueller, spokesm...|
| 18|    0|[snl, hilariously...|
| 19|    0|[republican, sena...|
+---+-----+--------------------+
only showing top 20 rows

+--------------------+--------------------+
|                  id|               token|
+--------------------+--------------------+
|6a175f46bcd24d39b...|[muslim, bust, st...|
|2bdc2

## CountVector
- train the count vector model (no need to run again)
- save the model (no need to run again)
- read the pre-trained model and generate the count vectors for the datasets

### Output: 
- label_data_cv (spark dataframe, column=['id', 'label', 'token', 'count_vector'])
- main_data_cv (spark dataframe, column=['id', 'token', 'count_vector'])

In [ ]:
# # generate the count vector for the labeled dataset
# cv = CountVectorizer(inputCol='token', outputCol='count_vector')
# m_cv = cv.fit(label_data_df)
# label_data_cv = m_cv.transform(label_data_df)
# label_data_cv.show()

time: 1.46 ms (started: 2022-03-20 11:32:57 +00:00)


In [ ]:
# # generate the count vector for the unlabeled dataset
# main_data_cv = m_cv.transform(main_data_df)
# main_data_cv.show(8, False)

time: 1.1 ms (started: 2022-03-20 11:32:57 +00:00)


In [ ]:
# save the trained countvectorizer model
modelPath = '/Users/yishun/Desktop/BAModule/CS5344/group project/Data Processing/count_vectorizer_model'
# m_cv.save(modelPath)

# generate the count vectors directly using the pre-trained model
loadedModel = CountVectorizerModel.load(modelPath)
label_data_cv = loadedModel.transform(label_data_df)
main_data_cv = loadedModel.transform(main_data_df)

label_data_cv.show()
main_data_cv.show()

+---+-----+--------------------+--------------------+
| id|label|               token|        count_vector|
+---+-----+--------------------+--------------------+
|  0|    0|[donald, trump, s...|(100949,[0,1,3,5,...|
|  1|    0|[drunk, bragging,...|(100949,[0,1,2,3,...|
|  2|    0|[sheriff, david, ...|(100949,[0,6,7,8,...|
|  3|    0|[trump, obsess, e...|(100949,[0,1,2,3,...|
|  4|    0|[pope, francis, c...|(100949,[0,1,2,4,...|
|  5|    0|[racist, alabama,...|(100949,[1,5,6,14...|
|  6|    0|[fresh, golf, cou...|(100949,[0,1,3,7,...|
|  7|    0|[trump, say, insa...|(100949,[0,1,3,4,...|
|  8|    0|[former, cia, dir...|(100949,[0,2,3,4,...|
|  9|    0|[watch, brand, ne...|(100949,[0,1,2,3,...|
| 10|    0|[papa, john, foun...|(100949,[0,2,5,6,...|
| 11|    0|[watch, paul, rya...|(100949,[0,1,2,4,...|
| 12|    0|[bad, news, trump...|(100949,[0,1,2,3,...|
| 13|    0|[watch, lindsey, ...|(100949,[0,1,3,9,...|
| 14|    0|[heiress, disney,...|(100949,[0,1,2,6,...|
| 15|    0|[tone, deaf, trum

## TF-IDF Vector
### Steps:
- use HashingTF to calculate the TF
- train the TF-IDF vector model (no need to run again)
- save the model (no need to run again)
- read the pre-trained model and generate the TF-IDF vectors for the datasets

### Output: 
- label_data_tf_idf (spark dataframe, column=['id', 'label', 'token', 'tf_vector', 'tf_idf_vector'])
- main_data_tf_idf (spark dataframe, column=['id', 'token', 'tf_vector', 'tf_idf_vector'])

In [ ]:
# generate the TF-IDF vector of the labeled dataset 
tf = HashingTF(inputCol='token', outputCol='tf_vector')
label_data_tf = tf.transform(label_data_df)
label_data_tf.show()

# tf_idf = IDF(inputCol='tf_vector', outputCol='tf_idf_vector')
# m_tf_idf = tf_idf.fit(label_data_tf)
# label_data_tf_idf = m_tf_idf.transform(label_data_tf)
# label_data_tf_idf.show()

+---+-----+--------------------+--------------------+
| id|label|               token|           tf_vector|
+---+-----+--------------------+--------------------+
|  0|    0|[donald, trump, s...|(262144,[531,1512...|
|  1|    0|[drunk, bragging,...|(262144,[3148,471...|
|  2|    0|[sheriff, david, ...|(262144,[531,794,...|
|  3|    0|[trump, obsess, e...|(262144,[531,654,...|
|  4|    0|[pope, francis, c...|(262144,[2437,392...|
|  5|    0|[racist, alabama,...|(262144,[1226,425...|
|  6|    0|[fresh, golf, cou...|(262144,[1004,151...|
|  7|    0|[trump, say, insa...|(262144,[2306,594...|
|  8|    0|[former, cia, dir...|(262144,[619,1004...|
|  9|    0|[watch, brand, ne...|(262144,[4214,853...|
| 10|    0|[papa, john, foun...|(262144,[2325,236...|
| 11|    0|[watch, paul, rya...|(262144,[531,1512...|
| 12|    0|[bad, news, trump...|(262144,[1451,160...|
| 13|    0|[watch, lindsey, ...|(262144,[531,1512...|
| 14|    0|[heiress, disney,...|(262144,[531,1512...|
| 15|    0|[tone, deaf, trum

In [ ]:
# generate the TF-IDF vector of the unlabeled dataset
main_data_tf = tf.transform(main_data_df)
# main_data_tf_idf = m_tf_idf.transform(main_data_tf)
# main_data_tf_idf.show()

time: 39.4 ms (started: 2022-03-20 11:33:28 +00:00)


In [ ]:
# save the trained IDF Model 
modelPath = '/Users/yishun/Desktop/BAModule/CS5344/group project/Data Processing/TF_IDF_model'
# m_tf_idf.save(modelPath)

# generate the TF-IDF vectors directly using the pre-trained model
loadedModel = IDFModel.load(modelPath)
label_data_tf_idf = loadedModel.transform(label_data_tf)
main_data_tf_idf = loadedModel.transform(main_data_tf)

label_data_tf_idf.show()
main_data_tf_idf.show()

+---+-----+--------------------+--------------------+--------------------+
| id|label|               token|           tf_vector|       tf_idf_vector|
+---+-----+--------------------+--------------------+--------------------+
|  0|    0|[donald, trump, s...|(262144,[531,1512...|(262144,[531,1512...|
|  1|    0|[drunk, bragging,...|(262144,[3148,471...|(262144,[3148,471...|
|  2|    0|[sheriff, david, ...|(262144,[531,794,...|(262144,[531,794,...|
|  3|    0|[trump, obsess, e...|(262144,[531,654,...|(262144,[531,654,...|
|  4|    0|[pope, francis, c...|(262144,[2437,392...|(262144,[2437,392...|
|  5|    0|[racist, alabama,...|(262144,[1226,425...|(262144,[1226,425...|
|  6|    0|[fresh, golf, cou...|(262144,[1004,151...|(262144,[1004,151...|
|  7|    0|[trump, say, insa...|(262144,[2306,594...|(262144,[2306,594...|
|  8|    0|[former, cia, dir...|(262144,[619,1004...|(262144,[619,1004...|
|  9|    0|[watch, brand, ne...|(262144,[4214,853...|(262144,[4214,853...|
| 10|    0|[papa, john, f

## Standard the Format of the Outputs
### Outputs: 
- BoW 1-gram of the labeled dataset: label_cv_output (spark dataframe, column=['id', 'features', 'label']) 
- BoW 1-gram of the unlabeled dataset: main_cv_output (spark dataframe, column=['id', 'features'])
- TF-IDF of the labeled dataset: label_tfidf_output (spark dataframe, column=['id', 'features', 'label'])
- TF-IDF of the unlabeled dataset: main_tfidf_output (spark dataframe, column=['id', 'features'])

In [ ]:
# Output
label_cv_output = label_data_cv.select('id', 'count_vector', 'label')
main_cv_output = main_data_cv.select('id', 'count_vector')
label_tfidf_output = label_data_tf_idf.select('id', 'tf_idf_vector', 'label')
main_tfidf_output = main_data_tf_idf.select('id', 'tf_idf_vector')

label_cv_output = label_cv_output.withColumnRenamed('count_vector', 'features')
main_cv_output = main_cv_output.withColumnRenamed('count_vector', 'features')
label_tfidf_output = label_tfidf_output.withColumnRenamed('tf_idf_vector', 'features')
main_tfidf_output = main_tfidf_output.withColumnRenamed('tf_idf_vector', 'features')

label_cv_output.show()

+---+--------------------+-----+
| id|            features|label|
+---+--------------------+-----+
|  0|(100949,[0,1,3,5,...|    0|
|  1|(100949,[0,1,2,3,...|    0|
|  2|(100949,[0,6,7,8,...|    0|
|  3|(100949,[0,1,2,3,...|    0|
|  4|(100949,[0,1,2,4,...|    0|
|  5|(100949,[1,5,6,14...|    0|
|  6|(100949,[0,1,3,7,...|    0|
|  7|(100949,[0,1,3,4,...|    0|
|  8|(100949,[0,2,3,4,...|    0|
|  9|(100949,[0,1,2,3,...|    0|
| 10|(100949,[0,2,5,6,...|    0|
| 11|(100949,[0,1,2,4,...|    0|
| 12|(100949,[0,1,2,3,...|    0|
| 13|(100949,[0,1,3,9,...|    0|
| 14|(100949,[0,1,2,6,...|    0|
| 15|(100949,[0,1,5,7,...|    0|
| 16|(100949,[0,3,4,6,...|    0|
| 17|(100949,[0,1,3,7,...|    0|
| 18|(100949,[0,1,3,4,...|    0|
| 19|(100949,[0,1,3,5,...|    0|
+---+--------------------+-----+
only showing top 20 rows

time: 6.06 s (started: 2022-03-20 11:35:12 +00:00)


In [ ]:
import pyspark.ml.feature as ft
import numpy as np 
import pandas as pd

import pandas as pd
from pyspark.sql import SparkSession
import pyspark.ml.tuning as tune

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression,RandomForestClassifier,DecisionTreeClassifier,GBTClassifier



# spark = SparkSession \
#     .builder \
#     .appName('my_first_app_name') \
#     .getOrCreate()

#Data Example, for testing the code
# columns=['ID','word1','word2','label']
# number = np.array([[1,1,0,1],[2,0,1,0],[3,0,1,0],[4,1,1,1],[1,1,0,1],[2,0,1,0],[3,0,1,0],[4,1,1,1],[1,1,0,1],[2,0,1,0],[3,0,1,0],[4,1,1,1],[1,1,0,1],[2,0,1,0],[3,0,1,0],[4,1,1,1]])
# pd_data = pd.DataFrame(number,columns=columns)
# spark_data=spark.createDataFrame(pd_data)
# spark_data.show()

# data = pd.read_csv('/Users/yishun/Desktop/BAModule/BT5126 Hands on with BA/5126gp/Final_Data.csv')
# data = data.rename(columns={'result':'label'})
# spark_data=spark.createDataFrame(data)
def Model_part(spark_data):
    '''Input: Spark Dataframe, Output: two list, first list contains 4 best model, second list are the validation AUC-score
    '''

    # 80% training set 20% test set
    train_set, test_set = spark_data.randomSplit([0.8,0.2],seed=42)

    # The input dataset
    transform_train_set = train_set.select('features','label')
    transform_test_set = test_set.select('features','label')

    # Initialize 4 models
    logistic = LogisticRegression(featuresCol='features',labelCol='label')
    decision_tree = DecisionTreeClassifier(featuresCol='features',labelCol='label')
    random_forest = RandomForestClassifier(featuresCol='features',labelCol='label')
    gbt = GBTClassifier(featuresCol='features',labelCol='label')

    # Set the parameter grids for them
    loggrid = tune.ParamGridBuilder()\
        .addGrid(logistic.maxIter, [5,10,50])\
        .addGrid(logistic.regParam, [0.1,0.01,0.05]) \
        .addGrid(logistic.elasticNetParam, [0.0, 0.5, 1.0])\
        .build()

    dtcgrid = tune.ParamGridBuilder()\
        .addGrid(decision_tree.maxDepth, [3,6,7,9])\
        .build()

    rfgrid = tune.ParamGridBuilder() \
        .addGrid(random_forest.numTrees,[100,200,300]) \
        .addGrid(random_forest.maxDepth, [2,3,4,5]) \
        .build()

    gbtgrid = tune.ParamGridBuilder() \
        .addGrid(gbt.maxDepth,[2,3,4]) \
        .addGrid(gbt.stepSize,[0.01,0.001,0.005]) \
        .build()

    # 5-fold Cross validation
    logcv = tune.CrossValidator(
        estimator=logistic,
        estimatorParamMaps=loggrid,
        evaluator=BinaryClassificationEvaluator(),
        numFolds=5
    )
    dtcv = tune.CrossValidator(
        estimator=decision_tree,
        estimatorParamMaps=dtcgrid,
        evaluator=BinaryClassificationEvaluator(),
        numFolds=5
    )
    rfcv = tune.CrossValidator(
        estimator=random_forest,
        estimatorParamMaps=rfgrid,
        evaluator=BinaryClassificationEvaluator(),
        numFolds=5
    )
    gbtcv = tune.CrossValidator(
        estimator=gbt,
        estimatorParamMaps=gbtgrid,
        evaluator=BinaryClassificationEvaluator(),
        numFolds=5
    )

    # Fit on the training set
    logmodel = logcv.fit(transform_train_set)
    dtmodel = dtcv.fit(transform_train_set)
    rfmodel = rfcv.fit(transform_train_set)
    gbtmodel = gbtcv.fit(transform_train_set)

    # Default ROC-AUC
    evaluator = BinaryClassificationEvaluator(rawPredictionCol= 'probability',labelCol='label')
    # Return all the best model
    best_model_list = [logmodel.bestModel,dtmodel.bestModel,rfmodel.bestModel,gbtmodel.bestModel]
    # Return
    auc_list = [evaluator.evaluate(model.transform(transform_test_set)) for model in best_model_list]

    return best_model_list,auc_list

In [ ]:
cv_model, cv_auc = Model_part(label_cv_output)

In [ ]:
tfidf_model, ifidf_auc = Model_part(label_tfidf_output)